# Introduction to Markov Chains in Python

Markov chains are an example of how simple ideas in probabilities can be extended to create sophisticated models that capture change over time in a dynamic system. Markov chains form the basis of some very powerful methods such as **Markov chain Monte Carlo (MCMC)** models used to generate samples from the posterior distribution in Bayesian models. Markov chains can also be further extended to create other models such as **hidden Markov models (HMMs)**.

There is a lot of theory to Markov chains, some of which addressed in the slides, but in this notebook we focus on a simple example to illustrate the idea.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import naive_bayes
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix

import matplotlib
matplotlib.rcParams['figure.figsize'] = [15, 5]

We will use the tennis data to run our example. For this example, we assume that the tennis data **is sorted by date and the rows correspond to consecutive days**. This is not necessarily true of this data set. It's just an assumption we make so we can illustrate Markov chains.

In [3]:
tennis = pd.read_csv('../../data/tennis.csv')
tennis.head()

,outlook,temp,humidity,windy,play
0,sunny,hot,high,strong,no
1,sunny,hot,high,weak,no
2,overcast,hot,high,weak,yes
3,rain,mild,high,weak,yes
4,rain,cool,normal,weak,yes


The only column we need here is `outlook`. Since we are assuming (for the sake of argument) that each row is a consecutive day, we can think of `outlook` as the outlook today and lag it by 1 day to get the outlook yesterday.

In [4]:
outlook = pd.concat([tennis['outlook'].shift(1), tennis['outlook']], axis = 1).dropna()
outlook.columns = ['yesterday', 'today']
outlook.head()

,yesterday,today
1,sunny,sunny
2,sunny,overcast
3,overcast,rain
4,rain,rain
5,rain,rain


We can now get counts from the above tables to see how the `outlook` transitions from yesterday to today.

In [5]:
pd.crosstab(outlook['yesterday'], outlook['today'])

today,overcast,rain,sunny
yesterday,,,
overcast,1,2,1
rain,1,2,1
sunny,2,1,2


If we turn the counts into percentages, we get our transition matrix. To do this, we can just use the `normalize = 1`, where 1 here refers to normalizing **by column**.

In [6]:
tr_mat = pd.crosstab(outlook['today'], outlook['yesterday'], normalize = 1)
tr_mat

yesterday,overcast,rain,sunny
today,,,
overcast,0.25,0.25,0.4
rain,0.50,0.50,0.2
sunny,0.25,0.25,0.4


### Exercise

- Let's say that today's weather is overcast. Using the above transition matrix, predict tomorrow's weather. In other words, given the weather today, what is the probability distribution of the weather tomorrow. HINT: To do this, first we need to think how we can encode the fact that today's weather is overcast, and then we need to perform a matrix multiplication using `np.dot`.

In [9]:
tom = np.dot(tr_mat, tr_mat['overcast'])
print(f"Tomorrow's weather by probability:\n\tovercast: {tom[0]}\n\train: {tom[1]}\n\tsunny: {tom[2]}")

Tomorrow's weather by probability:
	overcast: 0.2875
	rain: 0.425
	sunny: 0.2875


- Assuming that today's weather is overcast, now find the distribution of the weather 2 days from today?

In [12]:
day2 = np.dot(tr_mat,(np.dot(tr_mat, tr_mat["overcast"])))
print(f"Day after tomorrow's weather by probability:\n\tovercast: {day2[0]}\n\train: {day2[1]}\n\tsunny: {day2[2]}")

Day after tomorrow's weather by probability:
	overcast: 0.29312499999999997
	rain: 0.41374999999999995
	sunny: 0.29312499999999997


- Assuming that today's weather is overcast, now calculate and print the distribution of the weather over the next 10 days. How do the probabilities change over time? HINT: To code this in a clean way, it's best to avoid calling the matrix multiplication recursively. So instead, we write a loop to implement the recursion. We laid out a lot of the code here already. You just need to add two additional lines of code to implement this.

In [14]:
n_iter = 10 # number of days
A = [1, 0, 0] # weather today
probs = np.ones((n_iter, 3)) # a matrix to store the distributio of probabilites

for i in range(n_iter):
    ## store the distribution of weather in ith row of probs
    next = np.dot(tr_mat, A)
    ## update the distribution of weather
    probs[i,:] = next
    A = next
    

print(probs)

[[0.25       0.5        0.25      ]
 [0.2875     0.425      0.2875    ]
 [0.293125   0.41375    0.293125  ]
 [0.29396875 0.4120625  0.29396875]
 [0.29409531 0.41180937 0.29409531]
 [0.2941143  0.41177141 0.2941143 ]
 [0.29411714 0.41176571 0.29411714]
 [0.29411757 0.41176486 0.29411757]
 [0.29411764 0.41176473 0.29411764]
 [0.29411765 0.41176471 0.29411765]]


- Using the long term distribution of weather, we can also generate daily weather data. To see how, run the cell below multiple times and see how the results change. We are using the **multinomial distribution** to generate the samples here. This distribution is an extension of the **binomial distribution** to more than two outcomes. Sampling from it is like throwing a biased n-sided die where each side has its own probability of being rolled (if the die were fair, all probabilities would be equal to $1/n$).

In [ ]:
np.random.multinomial(1, A)

We should see that the probabilities seem to stabilize very quickly, and once stabilized, the system reaches some sort of equilibrium. Of course this only happens if certain conditions are met, which is addressed by the theory of Markov chains, but we will leave it here.

### End of exercise

There are many immediate applications to Markov chains. As one example, consider how a Markov chain model could be used to create a word completion algorithm or even a sentence completion algorithm. Of course such an algorithm would make some very simplistic assumptions and there are far better alternatives we can use these days, but many of the state-of-the-art algorithms today are also often based on simple ideas. Another thing to note is that because Markov chains are based on probabilities, once we get the long term probabilities we can "generate" new examples as we saw in the exercise. This means that Markov chains are also a "generative model", i.e. we can use them to generate new words or new sentences (not necessarily grammatically correct). Generative models are very useful in artificial intelligence.

# Introduction to naive Bayes models

This notebook introduces you to naive Bayes models. Naive Bayes models are a surprisingly useful and effective simplification of the general Bayesian models. Naive Bayes models make the naive assumption of independence of the features.

Some properties of naive Bayes models are:

- Computational complexity is linear in number of parameter / features
- Require minimal data to produce models that generalizes well
- Have a simple and inherent regularization

Naive Bayes models are widely used including for:

- Document classification
- SPAM detection
- Image classification

### Example - income prediction using Census data

Let's try a binary classification example on some sample US Census data. We want to build and evaluate a naive Bayes model to classify people by high and low income using $50,000 as the cut-off. Execute this code and examine the features in the data set. 

In [ ]:
census = pd.read_csv('../data/census.csv', sep = ',\\s+', engine = 'python')
census.head()

In [ ]:
census.info()

We can see some features which are likely to be collinear. There is also one feature, `fnlwgt`, which is not useful in classifying these people. We also convert all columns of type `object` into type `category`.

In [ ]:
census = census.drop(columns = ['workclass', 'fnlwgt', 'education-num', 'relationship'])
census[census.select_dtypes('object').columns] = census.select_dtypes('object').astype('category')
census.head()

Let's compute a naive Bayes model to classify `income` using the features in the Income data set. Since we have a mix of categorical and numeric features, we need to train two separate naive Bayes classifiers and then combine their results at the end.

In [ ]:
features_num = census.select_dtypes('int64')
features_cat = census.drop(columns = ['income']).select_dtypes('category').apply(lambda x: x.cat.codes)
# X = pd.concat([features_num, features_cat], axis = 1)
Y = census['income'].cat.codes

We can now train our classifier. Read [here](https://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes) about the hyper-parameters of the naive Bayes classifier. Since we have a mix of categorical and numeric features, we train a separate classifier on the categorical features and numeric features. We will later combine them.

In [ ]:
mnb = naive_bayes.CategoricalNB()
mnb.fit(features_cat, Y)

gnb = naive_bayes.GaussianNB()
gnb.fit(features_num, Y)

Although naive Bayes can be a good classifier, it is not necessarily a good estimator, so the soft predictions obtained from `predict_proba` should be viewed with caution. So we should not be too eager in interpreting the soft predictions as probabilities, but we can definitely compare them across the different categories to see each category's contribution to the final (posterior) probability. Here we can see that for the categories of `education`.

In [ ]:
dd_0 = pd.DataFrame({'cat': census['education'].cat.categories, 'prob': np.exp(mnb.feature_log_prob_[0][0])})
dd_1 = pd.DataFrame({'cat': census['education'].cat.categories, 'prob': np.exp(mnb.feature_log_prob_[0][1])})
dd_0['y'] = 'income is low'
dd_1['y'] = 'income is high'
dd = pd.concat([dd_0,dd_1], ignore_index=True)

# let's keep only the relevant categories and order them in a way that makes sense
cat_order = ['HS-grad', 'Prof-school', 'Assoc-voc', 'Assoc-acdm', 'Bachelors', 'Masters', 'Doctorate']
dd['cat'] = dd['cat'].astype('category').cat.set_categories(cat_order)

bar = sns.barplot(x = 'cat', y = 'prob', data = dd, hue = 'y');

For the numeric feature, we can try to visualize the results by looking at how the posterior probabilities change as a result of changing one variable (holding the others constant at their average values).

In [ ]:
age_min, age_max = features_num.describe()['age'][['min', 'max']]
age_range = np.linspace(age_min, age_max, num = 50)
dd = pd.DataFrame({'age': age_range})

dd['capital-gain'] = features_num['capital-gain'].mean()
dd['capital-loss'] = features_num['capital-loss'].mean()
dd['hours-per-week'] = features_num['hours-per-week'].mean()

sns.lineplot(x = age_range, y = gnb.predict_proba(dd)[:, 1]);

If we had only one model, then we can obtain hard predictions by calling its `predict` model and get soft predictions by calling `predict_prob`. We can also indirectly obtain hard predictions by using `np.argmax` on the soft predictions.

Since we need to combine both models into one, we need to do obtain hard predictions manually: 
- First we get soft predictions by taking the the product of each model's soft predictions (probabilities). Or in log terms, we add the log probabilites.
- Finally we use `np.argmax` to get hard predictions from the soft predictions.

In [ ]:
combined_preds = mnb.predict_log_proba(features_cat) + gnb.predict_log_proba(features_num)
combined_preds.shape
census['pred_income_high'] = np.argmax(combined_preds, axis = 1)

census.groupby('pred_income_high').mean(numeric_only=True)

Let's look at the model accuracy.

In [ ]:
accuracy_score(census['income'] == '>50K', census['pred_income_high'])

How good a model is this? We leave it to the reader to try other performance measure or even try another model like logistic regression for comparison.

# Assignment

Document classification has been one of the most successful applications of the naive Bayes model. There is a good chance that the SPAM filter your email service uses is a naive Bayes model, at least in part.

We say that we classify documents by **topics**. The naive Bayes topic model computes the probability that a document $D$ has topic $C$ based on the occurrence of the words $\{ w_1, w_2, \ldots, w_n \}$, using the following relationship:

$$p(C|D) \propto \prod_{j = 1}^N p(w_j|C)$$

Notice that this topic model allows a document to have a number of topics. For example, we can say the topics of $D$ are the 5 topics with the highest probability.

For a SPAM classifier, the topics are just spam and not spam, so we only need a Bernoulli topic model:

$$p(S+|D) \propto p(S+) \prod_{j=1}^N p(w_j|S+)$$

For this assignment we will use the `HouseVotes84` data which contains political party and votes on 16 important bills for 435 members of the US House of Representatives in 1984. We will use this data set to build and test a classifier to predict the political party of representatives.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import naive_bayes
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix

import matplotlib
matplotlib.rcParams['figure.figsize'] = [15, 5]

In [ ]:
vote_names = [
    'handicapped_infants',
    'water_project_cost_sharing',
    'adoption_of_the_budget_resolution',
    'physician_fee_freeze',
    'el_salvador_aid',
    'religious_groups_in_schools',
    'anti_satellite_test_ban',
    'aid_to_nicaraguan_contras',
    'mx_missile',
    'immigration',
    'synfuels_corporation_cutback',
    'education_spending',
    'superfund_right_to_sue',
    'crime',
    'duty_free_exports',
    'export_administration_act_south_africa']

votes = pd.read_csv('../data/house-votes-84.csv', 
                    header = None, names = ['party'] + vote_names)
print(votes.shape)
votes.head()

- Reshape the data from wide to long, so the new data called `votes_long` has only three columns: `party`, `issue` and `vote`. HINT: You can use the `melt` method to reshape the data. <span style="color:red" float:right>[5 point]</span>

In [ ]:
## you code goes here

- Visualize the data by looking at a barplot of the frequency of votes by party for each issue. HINT: Use `sns.catplot`. <span style="color:red" float:right>[5 point]</span>

In [ ]:
## you code goes here

- Return to the wide data `votes`: Loops though the columns `vote_names` and perform the following: <span style="color:red" float:right>[15 point]</span>
  - Convert the columns into type `category` and limit the categories to `y` and `n`. What happens with all the `?` votes?
  - If there are missing values in these columns, replace the missing values with the majority vote **by party affiliation**. HINT: To look at an example, run `votes.groupby('party')['immigration'].transform(lambda x: x.fillna(x.mode()[0]))`.
  - Replace a `yes` vote with 1 and a `no` vote with 0 (we do this because `sklearn` doesn't like string columns).

In [ ]:
# votes.groupby('party')['immigration'].value_counts()
# votes['immigration'].fillna(votes['immigration'].mode()[0])

In [ ]:
## you code goes here

Make sure that no NAs are left in the data. Time to train a classifier to predict party affiliation based on how someone votes on the above issues. The model will predict the probability of being a Democrat. For simplicity, we assume that the probability of being a Republican is 1 minus that of being a Democrat.

- Train a multinomial naive Bayes classifier to predict whether someone is a Democrat based on their vote. To keep things simple, you do NOT need to split the data into training and testing. <span style="color:red" float:right>[10 point]</span>

In [ ]:
## you code goes here

- Create a new column in the data called `pred_party` and store the model's predicted party affilation in it. Note that you will need to express the predictions in terms of party affiliation: `democrat` or `republican`. <span style="color:red" float:right>[10 point]</span>

In [ ]:
## you code goes here

- Get the accuracy and confusion matrix. <span style="color:red" float:right>[5 point]</span>

In [ ]:
## you code goes here

# End of assignment